In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

!pip install -q imageio
!pip install -q opencv-python
!pip install -q git+https://github.com/tensorflow/docs

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import *
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
#from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import os
import math

# TensorFlow and TF-Hub modules.
#from absl import logging
#import tensorflow_hub as hub
#from tensorflow_docs.vis import embed

#logging.set_verbosity(logging.ERROR)

# Some modules to help with reading the UCF101 dataset.
import random
import re
import tempfile
import ssl

# Some modules to display an animation using imageio.
#import imageio
#from IPython import display

#from urllib import request  # requires python3

In [30]:
image_size= 64 # ie. 64*64

#number of frames to passs at a time in a sequence
sequence_len = 10

#data directoy
dataset_dir="../input/ucf101/UCF101/UCF-101"

#selected classes
calss_names_path = "../input/ucf101/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/classInd.txt"#

# class_list stores all the 101 classes
#class_list = []
#with open(calss_names_path,"r") as f:
#  for line in f:
#    class_list.append(line.split()[1])
#
#f.close()

#print(class_list)

#subset list that we wil work on
selected_class_list = ["TaiChi","HorseRace"]

In [6]:
# Helper functions

#crops center portioin of frame
def crop_center_square(frame):
  y, x = frame.shape[0:2]
  min_dim = min(y, x)
  start_x = (x // 2) - (min_dim // 2)
  start_y = (y // 2) - (min_dim // 2)
  return frame[start_y:start_y+min_dim,start_x:start_x+min_dim]

######################################################################################  

# returns normalized numpy array of each frame(image) in video(sequence of images) 
def extract_frames(path, max_frames=0, resize=(224, 224)):
  cap = cv2.VideoCapture(path)
  frames_list = []
  #frameRate = cap.get(5) #frame rate 
  video_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  skip_frame_window = max(int(video_frame_count / sequence_len),1)
  
  try:
    for frame_counter in range(sequence_len):
      cap.set(cv2.CAP_PROP_POS_FRAMES, frame_counter*skip_frame_window) #current frame number
      
      ret, frame = cap.read()
      
      if not ret:
        break

      #if (frameId % math.floor(frameRate) == 0):
      croped_frame = crop_center_square(frame) #crop frame
      resized_frame = cv2.resize(croped_frame, resize) #resize frame
      frame = frame[:, :, [2, 1, 0]]
      frames_list.append(resized_frame)

      if len(frames_list ) == max_frames:
        break
  finally:
    cap.release()
  #print("[INFO] {} Frames extracted".format(len(frames_list)))
  # return normalized frames list 
  return np.array(frames_list ) / 255.0

######################################################################################  


#def to_gif(images):
  #converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
  #imageio.mimsave('./animation.gif', converted_images, fps=25)
  #return embed.embed_file('./animation.gif')

######################################################################################  



# Create Dataset

In [31]:
def create_dataset():
  features=[] #list of multiple lists of frames from one video class (X)
  lables=[] #list of class lables (y)
  video_file_paths=[]

  for class_index, class_name in enumerate(selected_class_list):
    print("[INFO] Exgtracting data of class: {}".format(class_name))

    #list of all video files in one class directory
    files_list = os.listdir(os.path.join(dataset_dir, class_name))

    #iterating through all files in class
    for file_name in files_list:
      #get complete video path
      video_file_path = os.path.join(dataset_dir, class_name, file_name)

      #extracting frames of the video
      frames = extract_frames(video_file_path,resize=(image_size,image_size))
      
      #select if no. of frames match sequence length else dont select
     # if len(frames) == sequence_len:
      features.append(frames)
      lables.append(class_index)
      video_file_paths.append(video_file_path)
  return features, lables, video_file_paths         

In [32]:
features, labels, video_file_paths = create_dataset()

[INFO] Exgtracting data of class: TaiChi
[INFO] Exgtracting data of class: HorseRace


In [33]:
# convert the data and labels to NumPy arrays
features = np.array(features)
labels = np.array(labels)
# perform one-hot encoding on the labels
one_hot_encoded_labels = to_categorical(labels)
#lb = LabelBinarizer()
#labels = lb.fit_transform(labels)
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(features, labels,
	test_size=0.25,shuffle=True, stratify=labels, random_state=42)

In [34]:
trainX.shape

(168, 10, 64, 64, 3)

In [41]:
def create_cnnlstm_model():
  model=Sequential()

  model.add(ConvLSTM2D(filters=4, kernel_size=(3,3), activation='tanh', data_format='channels_last', 
                       recurrent_dropout=0.2, return_sequences=True, input_shape = (sequence_len,image_size,image_size,3)))
  model.add(MaxPooling3D(pool_size=(1,2,2), padding='same', data_format='channels_last'))
  model.add(TimeDistributed(Dropout(0.2)))
  

  model.add(ConvLSTM2D(filters=8, kernel_size=(3,3), activation='tanh', data_format='channels_last', 
                       recurrent_dropout=0.2, return_sequences=True))
  model.add(MaxPooling3D(pool_size=(1,2,2), padding='same', data_format='channels_last'))
  model.add(TimeDistributed(Dropout(0.2)))
  
  
  model.add(ConvLSTM2D(filters=14, kernel_size=(3,3), activation='tanh', data_format='channels_last', 
                       recurrent_dropout=0.2, return_sequences=True))
  model.add(MaxPooling3D(pool_size=(1,2,2), padding='same', data_format='channels_last'))
  model.add(TimeDistributed(Dropout(0.2)))
  
  
  model.add(ConvLSTM2D(filters=16, kernel_size=(3,3), activation='tanh', data_format='channels_last', 
                       recurrent_dropout=0.2, return_sequences=True))
  model.add(MaxPooling3D(pool_size=(1,2,2), padding='same', data_format='channels_last'))
  #model.add(TimeDistributed(Dropout(0.2)))

  model.add(Flatten())

  model.add(Dense(len(selected_class_list), activation='softmax'))

  model.summary

  return model

In [42]:
convlstm_model= create_cnnlstm_model()
print("model created")

model created


In [43]:
epochs=30
batch_size=5

In [46]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True )
convlstm_model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

model_training = convlstm_model.fit(x=trainX, y=trainY, epochs=epochs, batch_size=batch_size,
                                    shuffle=True, validation_split=0.2, callbacks=[early_stopping_callback])

Epoch 1/30
27/27 [==============================] - 32s 730ms/step - loss: 0.6975 - accuracy: 0.4733 - val_loss: 0.6896 - val_accuracy: 0.6471
Epoch 2/30
27/27 [==============================] - 18s 677ms/step - loss: 0.6889 - accuracy: 0.6292 - val_loss: 0.6571 - val_accuracy: 0.5588
Epoch 3/30
27/27 [==============================] - 18s 684ms/step - loss: 0.5699 - accuracy: 0.7290 - val_loss: 0.5063 - val_accuracy: 0.7353
Epoch 4/30
27/27 [==============================] - 18s 675ms/step - loss: 0.4281 - accuracy: 0.8539 - val_loss: 0.4372 - val_accuracy: 0.7941
Epoch 5/30
27/27 [==============================] - 19s 689ms/step - loss: 0.4169 - accuracy: 0.8012 - val_loss: 0.7446 - val_accuracy: 0.6471
Epoch 6/30
27/27 [==============================] - 18s 682ms/step - loss: 0.4397 - accuracy: 0.8576 - val_loss: 0.5514 - val_accuracy: 0.6471
Epoch 7/30
27/27 [==============================] - 18s 685ms/step - loss: 0.5037 - accuracy: 0.7887 - val_loss: 0.7178 - val_accuracy: 0.7353

In [47]:
model_evaluation = convlstm_model.evaluate(testX,testY)

2/2 [==============================] - 2s 723ms/step - loss: 0.2405 - accuracy: 0.8393
